# Logical Agents

 This notebook serves as a supporting material for the chapter **Logical Agents**. The notebooks illustrate the use of the code repository and demonstrate how the code can be extended to solve various related problems. We begin with overall agent design, introduce a simple new environment, the wumpus world, and illustrate the operation of knowledge-based agent. Then we have a look at the gernal principles of **logic** and the specifics of **propositional logic** and with well-developed inference technologies. At last, we combine the concept of knowledge-based agents with technology of propositional logic to build some simple agents for the wumpus world.

## Knowledge-Based Agents

The central component of a knowledge-based agent is its **knowledge base**, or **KB**. A knowledge base is a set of **sentences**. (Here “sentence” is used as a technical term. It is related but not identical to the sentences of English and other natural languages.) Each sentence is expressed in a language called a **knowledge representation language** and represents some assertion about the world. Sometimes we dignify a sentence with the name **axiom**, when the
sentence is taken as given without being derived from other sentences.

There must be a way to add new sentences to the knowledge base and a way to query what is known. The standard names for these operations are TELL and ASK, respectively.Both operations may involve **inference**—that is, deriving new sentences from old. Inference must obey the requirement that when one ASKs a question of the knowledge base, the answer should follow from what has been told (or TELLed) to the knowledge base previously.

The pseudocode below shows the outline of a knowledge-based agent program. Like all other agents, it takes a percept as input and returns an action. The agent maintaining knowlege base, KB, which may initially contain some **background knowledge*

In [1]:
from notebookUtils import *
pseudocode('KB-Agent')

### AIMA3e
__function__ KB-AGENT(_percept_) __returns__ an _action_  
&emsp;__persistent__: _KB_, a knowledge base  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;_t_, a counter, initially 0, indicating time  

&emsp;TELL(_KB_, MAKE\-PERCEPT\-SENTENCE(_percept_, _t_))  
&emsp;_action_ &larr; ASK(_KB_, MAKE\-ACTION\-QUERY(_t_))  
&emsp;TELL(_KB_, MAKE\-ACTION\-SENTENCE(_action_, _t_))  
&emsp;_t_ &larr; _t_ + 1  
&emsp;__return__ _action_  

---
__Figure__ ?? A generic knowledge\-based agent. Given a percept, the agent adds the percept to its knowledge base, asks the knowledge base for the best action, and tells the knowledge base that it has in fact taken that action.

The implementation of the above pseudocode can be viewed [here](https://github.com/aimacode/aima-java/blob/AIMA3e/aima-core/src/main/java/aima/core/logic/propositional/agent/KBAgent.java). This agent is implemented as an abstract agent which can be extended to construct other agents.